In [4]:
import os
import cv2
import numpy as np
#from google.colab.patches import cv2_imshow
import zipfile
import json
import pathlib
import labelme2coco
import shutil
import json
import os
import shutil
import yaml
import matplotlib.pyplot as plt

In [ ]:

# def Unzip_Data(path):
#   # unzip the downloaded file
#   zip_ref = zipfile.ZipFile(path)
#   zip_ref.extractall()
#   zip_ref.close()

#   return

In [5]:
import shutil

def delete_folder(folder_path):
    try:
        shutil.rmtree(folder_path)
        print(f"Folder '{folder_path}' and all its contents have been successfully deleted.")
    except Exception as e:
        print(f"Error deleting folder '{folder_path}': {e}")

# Example usage:
# folder_to_delete = "/content/Dataset"  # Replace this with the path of the folder you want to delete
# delete_folder(folder_to_delete)


In [6]:
from tkinter import Tk
from tkinter.filedialog import askdirectory

# Hide the main Tkinter window
Tk().withdraw()
# Print the selected folder
print(f"Please select the folder from the pop up, it will appear after you minimize this window:")
# Ask the user to select the folder with custom text
home_folder = askdirectory(title="Select the Project_submission folder")

# Check if a folder was selected
if not home_folder:
    print("No folder selected. Exiting.")
    exit()

# Print the selected folder or use it in your script
print(f"Selected folder: {home_folder}")


Please select the folder from the pop up, it will appear after you minimize this window:
Selected folder: D:/Others/Project_Submission_ALL_DATA


### 📁 copy_specific_files(input_folder, output_folder, extensions)

Copies all files with given extensions from a source folder (and its subfolders) into a single output folder.

**Parameters:**
- `input_folder` *(str)* – Path to the folder to search in.  
- `output_folder` *(str)* – Path to save copied files. Created if missing.  
- `extensions` *(set[str])* – File extensions to copy (e.g. `{'.png', '.json'}`).

**Example:**
```python
copy_specific_files(
    input_folder="Dataset/LableMe format",
    output_folder="Output_Folder",
    extensions={'.json', '.png'}
)


In [7]:
import os
import shutil

def copy_specific_files(input_folder, output_folder, extensions):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Walk through the input folder
    for root, _, files in os.walk(input_folder):
        for file in files:
            # Check if the file has one of the specified extensions
            _, file_extension = os.path.splitext(file)
            if file_extension.lower() in extensions:
                # Get the full path of the input file
                input_file_path = os.path.join(root, file)

                # Copy the file directly to the output folder without preserving subfolders
                shutil.copy(input_file_path, output_folder)

# Example usage:
input_folder = os.path.join(home_folder,"Dataset/Dataset containing annotation in LableMe format") # Change this to your input folder path
output_folder = "Output_Folder"  # Change this to your desired output folder path
extensions = {'.json', '.png', '.JSON', '.PNG'}  # Specify the extensions to copy
copy_specific_files(input_folder, output_folder, extensions)


### 📊 count_files_and_matching_pairs(folder_path)

Counts the number of `.json` and `.png` files in a folder and finds how many share the same base name.

**Parameters:**
- `folder_path` *(str)* – Path to the folder to check.

**Returns:**
`(json_count, png_count, matching_pairs)` – Counts of JSON files, PNG files, and matched name pairs.

**Example:**
```python
json_count, png_count, pairs = count_files_and_matching_pairs("Output_Folder")
print(json_count, png_count, pairs)


In [8]:
import os

def count_files_and_matching_pairs(folder_path):
    json_files = []
    png_files = []
    matching_pairs = 0

    # Iterate through the files in the folder
    for file_name in os.listdir(folder_path):
        # Check if it's a JSON file
        if file_name.lower().endswith('.json'):
            json_files.append(file_name)
        # Check if it's a PNG file
        elif file_name.lower().endswith('.png'):
            png_files.append(file_name)

    # Count matching pairs
    for png_file in png_files:
        # Generate the expected JSON file name
        json_file = os.path.splitext(png_file)[0] + '.json'
        # Check if the corresponding JSON file exists
        if json_file in json_files:
            matching_pairs += 1

    return len(json_files), len(png_files), matching_pairs

# Example usage:
folder_path = "Output_Folder"  # Change this to the path of your folder
json_count, png_count, matching_pairs = count_files_and_matching_pairs(folder_path)
print(f"Number of JSON files: {json_count}")
print(f"Number of PNG files: {png_count}")
print(f"Number of matching pairs (PNG file with corresponding JSON file): {matching_pairs}")


Number of JSON files: 1559
Number of PNG files: 1559
Number of matching pairs (PNG file with corresponding JSON file): 1559


- Ensures corresponding `.json` and `.png` files (same basename) stay together.  
- Randomly assigns each pair to either the training or validation set.  
- Avoids duplicate processing of files.

**Example:**
```python
split_train_validation(
  input_folder="Output_Folder",
  output_folder="Temp_Folder",
  split_ratio=0.8
)


In [9]:
import os
import random
import shutil

def split_train_validation(input_folder, output_folder, split_ratio=0.8):
    # Create train and validation folders inside the output folder
    train_folder = os.path.join(output_folder, 'train')
    validation_folder = os.path.join(output_folder, 'validation')
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(validation_folder, exist_ok=True)

    # Create 'images' and 'labels' folders inside both train and validation folders
    train_images_folder = os.path.join(train_folder, 'images')
    train_labels_folder = os.path.join(train_folder, 'labels')
    validation_images_folder = os.path.join(validation_folder, 'images')
    validation_labels_folder = os.path.join(validation_folder, 'labels')
    os.makedirs(train_images_folder, exist_ok=True)
    os.makedirs(train_labels_folder, exist_ok=True)
    os.makedirs(validation_images_folder, exist_ok=True)
    os.makedirs(validation_labels_folder, exist_ok=True)

    # Track processed file names to avoid duplication
    processed_files = set()

    # Walk through the input folder
    for root, _, files in os.walk(input_folder):
        for file in files:
            # Check if the file is either .json or .png
            if file.endswith('.json') or file.endswith('.PNG'):
                # Check if the file has already been processed to avoid duplication
                if file not in processed_files:
                    # Get the base name of the file (without extension)
                    base_name, _ = os.path.splitext(file)

                    # List both .json and .png files with the same base name
                    matching_files = [f for f in files if f.startswith(base_name)]

                    # Decide whether to put the files in train or validation set
                    if random.random() < split_ratio:
                        output_images_folder = train_images_folder
                        output_labels_folder = train_labels_folder
                    else:
                        output_images_folder = validation_images_folder
                        output_labels_folder = validation_labels_folder

                    # Copy both .json and .png files to the appropriate folders
                    for matching_file in matching_files:
                        input_file_path = os.path.join(root, matching_file)
                        output_file_path = os.path.join(output_labels_folder if matching_file.endswith('.json') else output_images_folder, matching_file)
                        shutil.copy(input_file_path, output_file_path)

                    # Add processed files to the set
                    processed_files.update(matching_files)

# Example usage:
input_folder = "Output_Folder"  # Change this to your input folder path
output_folder = "Temp_Folder"  # Change this to your desired output folder path
split_train_validation(input_folder, output_folder)


# 🧩 LabelMe to COCO Conversion and Image Copying (Validation Set)

This script converts LabelMe annotation files to COCO format for the validation dataset and copies corresponding images to the destination folder.

## Functionality
1. Converts annotations in LabelMe JSON format located in `Temp_Folder/validation` to COCO JSON format using `labelme2coco`.
2. Copies all `.png` image files from `Temp_Folder/validation/images` to `Input_folder/validation`.



In [10]:
import os
import shutil
import labelme2coco

# Convert LabelMe annotations to COCO format
labelme2coco.convert("Temp_Folder/train", "Input_folder/train/")

# List all images in the Temp_Folder/train/images directory
image_dir = os.path.join("Temp_Folder", "train", "images")
dir_list = os.listdir(image_dir)

# Copy all image files to Input_folder/train
output_dir = os.path.join("Input_folder", "train")
os.makedirs(output_dir, exist_ok=True)

for file in dir_list:
    src = os.path.join(image_dir, file)
    dst = os.path.join(output_dir, file)
    shutil.copy(src, dst)



There are 1276 listed files in folder Temp_Folder/train.


Converting labelme annotations to COCO format: 100%|██████████| 1276/1276 [00:40<00:00, 31.54it/s]
10/25/2025 00:35:50 - INFO - labelme2coco -   Converted annotations in COCO format is exported to Input_folder\train\dataset.json


In [11]:
# Convert LabelMe annotations to COCO format
labelme2coco.convert("Temp_Folder/validation", "Input_folder/validation/")

# List all images in the Temp_Folder/validation/images directory
image_dir = os.path.join("Temp_Folder", "validation", "images")
dir_list = os.listdir(image_dir)

# Copy all image files to Input_folder/validation
output_dir = os.path.join("Input_folder", "validation")
os.makedirs(output_dir, exist_ok=True)

for file in dir_list:
    src = os.path.join(image_dir, file)
    dst = os.path.join(output_dir, file)
    shutil.copy(src, dst)


There are 299 listed files in folder Temp_Folder/validation.


Converting labelme annotations to COCO format: 100%|██████████| 299/299 [00:09<00:00, 32.92it/s]
10/25/2025 00:40:36 - INFO - labelme2coco -   Converted annotations in COCO format is exported to Input_folder\validation\dataset.json


# 🖼️ Convert Images and Annotations to YOLO Format

This function converts images and their corresponding annotations from a JSON file to *YOLO format*

# Functionality

1. *Input Parameters:*
   - `input_images_path`: Folder containing the original images.
   - `input_json_path`: JSON file with image annotations.
   - `output_images_path`: Destination folder for YOLO-formatted images.
   - `output_labels_path`: Destination folder for YOLO-formatted label files.

2. *Output:*
   - Images copied to `output_images_path`.
   - YOLO label files generated in `output_labels_path`.


In [12]:


# Function to convert images to YOLO format
def convert_to_yolo(input_images_path, input_json_path, output_images_path, output_labels_path):
    # Open JSON file containing image annotations
    f = open(input_json_path)
    data = json.load(f)
    f.close()

    # Create directories for output images and labels
    os.makedirs(output_images_path, exist_ok=True)
    os.makedirs(output_labels_path, exist_ok=True)

    # List to store filenames
    file_names = []
    for filename in os.listdir(input_images_path):
        print(filename)

        if filename.endswith(".PNG"):

            source = os.path.join(input_images_path, filename)
            destination = os.path.join(output_images_path, filename)
            shutil.copy(source, destination)
            file_names.append(filename)
    print(file_names)
    # Function to get image annotations
    def get_img_ann(image_id):

        return [ann for ann in data['annotations'] if ann['image_id'] == image_id]

    # Function to get image data
    def get_img(filename):

        return next((img for img in data['images'] if img['file_name'].split('\\')[-1] == filename), None)

    # Iterate through filenames and process each image
    for filename in file_names:

        img = get_img(filename)
        img_id = img['id']
        img_w = img['width']
        img_h = img['height']
        img_ann = get_img_ann(img_id)


        # Write normalized polygon data to a text file
        if img_ann:

            with open(os.path.join(output_labels_path, f"{os.path.splitext(filename)[0]}.txt"), "a") as file_object:

                for ann in img_ann:

                    current_category = ann['category_id']

                    polygon = ann['segmentation'][0]

                    normalized_polygon = [format(coord / img_w if i % 2 == 0 else coord / img_h, '.6f') for i, coord in enumerate(polygon)]

                    file_object.write(f"{current_category} " + " ".join(normalized_polygon) + "\n")



# 🗂️ Create YAML File for Dataset

This function generates a **YAML file** for a dataset in a format compatible with YOLO.

## Functionality

1. **Input Parameters:**
   - `input_json_path`: Path to the JSON file containing dataset annotations and category information.
   - `output_yaml_path`: Path where the YAML file will be saved.
   - `train_path`: Path to the training dataset folder.
   - `val_path`: Path to the validation dataset folder.
   - `test_path` (optional): Path to the test dataset folder (defaults to empty string if not provided).


2. **Output:**
   - YAML file saved at `output_yaml_path` with the dataset structure ready for YOLO training.


In [13]:

# Function to create a YAML file for the dataset
def create_yaml(input_json_path, output_yaml_path, train_path, val_path, test_path=None):
    with open(input_json_path) as f:
        data = json.load(f)

    # Extract the category names
    names = [category['name'] for category in data['categories']]

    # Number of classes
    nc = len(names)

    # Create a dictionary with the required content
    yaml_data = {
        'names': names,
        'nc': nc,
        'test': test_path if test_path else '',
        'train': train_path,
        'val': val_path
    }

    # Write the dictionary to a YAML file
    with open(output_yaml_path, 'w') as file:
        yaml.dump(yaml_data, file, default_flow_style=False)


In [14]:
convert_to_yolo("Input_folder/train" , "Input_folder/train/dataset.json" , "Dataset/train/images" , "Dataset/train/labels")


dataset.json
IMG_0053.PNG
IMG_0054.PNG
IMG_0055.PNG
IMG_0056.PNG
IMG_0057.PNG
IMG_0058.PNG
IMG_0059.PNG
IMG_0060.PNG
IMG_0063.PNG
IMG_0065.PNG
IMG_0067.PNG
IMG_0068.PNG
IMG_0069.PNG
IMG_0071.PNG
IMG_0072.PNG
IMG_0073.PNG
IMG_0074.PNG
IMG_0075.PNG
IMG_0076.PNG
IMG_0077.PNG
IMG_0078.PNG
IMG_0079.PNG
IMG_0080.PNG
IMG_0081.PNG
IMG_0083.PNG
IMG_0086.PNG
IMG_0087.PNG
IMG_0088.PNG
IMG_0090.PNG
IMG_0091.PNG
IMG_0092.PNG
IMG_0093.PNG
IMG_0094.PNG
IMG_0095.PNG
IMG_0098.PNG
IMG_0100.PNG
IMG_0101.PNG
IMG_0103.PNG
IMG_0105.PNG
IMG_0107.PNG
IMG_0108.PNG
IMG_0111.PNG
IMG_0112.PNG
IMG_0113.PNG
IMG_0114.PNG
IMG_0115.PNG
IMG_0116.PNG
IMG_0117.PNG
IMG_0118.PNG
IMG_0119.PNG
IMG_0121.PNG
IMG_0123.PNG
IMG_0124.PNG
IMG_0126.PNG
IMG_0129.PNG
IMG_0130.PNG
IMG_0132.PNG
IMG_0133.PNG
IMG_0135.PNG
IMG_0136.PNG
IMG_0140.PNG
IMG_0141.PNG
IMG_0143.PNG
IMG_0144.PNG
IMG_0145.PNG
IMG_0146.PNG
IMG_0148.PNG
IMG_0149.PNG
IMG_0150.PNG
IMG_0152.PNG
IMG_0153.PNG
IMG_0154.PNG
IMG_0155.PNG
IMG_0157.PNG
IMG_0159.PNG
IMG_0160.PNG

In [21]:
import os
import json

input_json_path = "Input_folder/validation/dataset.json"

# Check if file exists first
if not os.path.exists(input_json_path):
    raise FileNotFoundError(f"File not found: {input_json_path}")

# Try to open and load JSON
try:
    with open(input_json_path, 'r') as f:
        dataset_json = json.load(f)
except json.JSONDecodeError as e:
    raise ValueError(f"Error decoding JSON in file {input_json_path}: {e}")
except Exception as e:
    raise RuntimeError(f"Unexpected error loading JSON file {input_json_path}: {e}")

print(f"Successfully loaded JSON. Type: {type(dataset_json)}")


Successfully loaded JSON. Type: <class 'dict'>


In [22]:
convert_to_yolo("Input_folder/validation" , "Input_folder/validation/dataset.json" , "Dataset/val/images" , "Dataset/val/labels")


dataset.json
IMG_0061.PNG
IMG_0066.PNG
IMG_0070.PNG
IMG_0084.PNG
IMG_0085.PNG
IMG_0089.PNG
IMG_0106.PNG
IMG_0109.PNG
IMG_0110.PNG
IMG_0120.PNG
IMG_0122.PNG
IMG_0125.PNG
IMG_0127.PNG
IMG_0128.PNG
IMG_0131.PNG
IMG_0137.PNG
IMG_0156.PNG
IMG_0158.PNG
IMG_0168.PNG
IMG_0173.PNG
IMG_0183.PNG
IMG_0188.PNG
IMG_0193.PNG
IMG_0200.PNG
IMG_0221.PNG
IMG_0236.PNG
IMG_0237.PNG
IMG_0241.PNG
IMG_0247.PNG
IMG_0253.PNG
IMG_0255.PNG
IMG_0257.PNG
IMG_0260.PNG
IMG_0262.PNG
IMG_0263.PNG
IMG_0276.PNG
IMG_0279.PNG
IMG_0281.PNG
IMG_0282.PNG
IMG_0287.PNG
IMG_0293.PNG
IMG_0297.PNG
IMG_0301.PNG
IMG_0310.PNG
IMG_0311.PNG
IMG_0313.PNG
IMG_10006.PNG
IMG_10008.PNG
IMG_10026.PNG
IMG_10035.PNG
IMG_10039.PNG
IMG_10054.PNG
IMG_10057.PNG
IMG_10064.PNG
IMG_10067.PNG
IMG_10092.PNG
IMG_10100.PNG
IMG_10108.PNG
IMG_10109.PNG
IMG_10112.PNG
IMG_10113.PNG
IMG_10117.PNG
IMG_10119.PNG
IMG_10120.PNG
IMG_10123.PNG
IMG_10127.PNG
IMG_10133.PNG
IMG_10136.PNG
IMG_10149.PNG
IMG_10150.PNG
IMG_10154.PNG
IMG_10156.PNG
IMG_10159.PNG
IMG_10165.P

In [27]:
create_yaml(
    input_json_path="Input_folder/train/dataset.json",
    output_yaml_path="Dataset/data.yaml",
    train_path=r"D:\Others\Project_Submission_ALL_DATA\MODEL TRAINING\Yolo_training\Dataset\train",
    val_path=r"D:\Others\Project_Submission_ALL_DATA\MODEL TRAINING\Yolo_training\Dataset\val"
)


## 🧠 YOLOv8 Model Training

This snippet demonstrates how to train a YOLOv8 segmentation model using a custom dataset defined in a YAML file.



In [ ]:
Root_dir = "Dataset"
from ultralytics import YOLO
# Load Model
model = YOLO("yolov8m-seg.pt")

#use the model
results = model.train(data = os.path.join(Root_dir ,"data.yaml") , epochs = 100)